<a href="https://colab.research.google.com/github/LuisRodriguez01d/TFM-RealState2022-LR/blob/main/TMF_Realstate_2022_Scrapper_Idealista_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# if scrapy no installed
!pip install scrapy

     |████████████████████████████████| 264 kB 5.1 MB/s 
     |████████████████████████████████| 251 kB 44.8 MB/s 
     |████████████████████████████████| 3.6 MB 44.1 MB/s 
     |████████████████████████████████| 87 kB 5.4 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 47 kB 4.1 MB/s 
     |████████████████████████████████| 3.1 MB 38.1 MB/s 
     |████████████████████████████████| 74 kB 2.2 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=11516 sha256=111250cb4dc69d90729c0e6ee7791255fe9936912eef6707becbe0151ba0c0ce
  Stored in directory: /root/.cache/pip/wheels/2d/18/21/3c6a732eaa69a339198e08bb63b7da2c45933a3428b29ec454
Successfully built PyDispatcher


In [ ]:
# Packages
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
import json
import datetime


In [ ]:
#load postcodes.txt
from google.colab import files
uploaded = files.upload()
!ls -la

Saving postcodes.txt to postcodes.txt
total 20
drwxr-xr-x 1 root root 4096 Mar 20 14:26 .
drwxr-xr-x 1 root root 4096 Mar 20 11:58 ..
drwxr-xr-x 1 root root 4096 Mar  9 14:47 .config
-rw-r--r-- 1 root root  390 Mar 20 14:26 postcodes.txt
drwxr-xr-x 1 root root 4096 Mar  9 14:48 sample_data


In [ ]:
# idealista scraper class
class CommercialSale(scrapy.Spider):
    #scraper name
    name = 'idealist'

    # base URL
    base_url = 'https://www.idealista.com/buscar/venta-viviendas/'
    # custom headers
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'es-ES,es;q=0.9',
        'cache-control': 'max-age=0',
        #'cookie': 'userUUID=3cd82b75-5416-43da-93c3-f29834bf2c58; contactf429c567-b38f-4aca-982d-01bcbf88d0c8="{'email':null,'phone':null,'phonePrefix':null,'friendEmails':null,'name':null,'message':null,'message2Friends':null,'maxNumberContactsAllow':10,'defaultMessage':true}"; SESSION=aef06c73bc8de1dd~f429c567-b38f-4aca-982d-01bcbf88d0c8; utag_main=v_id:017fa60afed00011a6e36c0cfd5905072003706a0086e$_sn:1$_se:1$_ss:1$_st:1647760012818$ses_id:1647758212818%3Bexp-session$_pn:1%3Bexp-session$_prevVtSource:directTraffic%3Bexp-1647761813195$_prevVtCampaignCode:%3Bexp-1647761813195$_prevVtDomainReferrer:%3Bexp-1647761813195$_prevVtSubdomaninReferrer:%3Bexp-1647761813195$_prevVtUrlReferrer:%3Bexp-1647761813195$_prevVtCampaignLinkName:%3Bexp-1647761813195$_prevVtCampaignName:%3Bexp-1647761813195$_prevVtRecommendationId:%3Bexp-1647761813195$_prevCompletePageName:16%3A%3Alisting%3A%3AresultList%3A%3Aothers%3A%3Ahttps%3A%2F%2Fwww.idealista.com%2Fbuscar%2Fventa-viviendas%2F28001%2F%3Bexp-1647761813196$_prevLevel2:16%3Bexp-1647761813196$_prevCompleteClickName:; atuserid=%7B%22name%22%3A%22atuserid%22%2C%22val%22%3A%221486e12f-d085-484d-9ab1-63c2ff0b1b38%22%2C%22options%22%3A%7B%22end%22%3A%222023-04-21T06%3A36%3A53.354Z%22%2C%22path%22%3A%22%2F%22%7D%7D; atidvisitor=%7B%22name%22%3A%22atidvisitor%22%2C%22val%22%3A%7B%22vrn%22%3A%22-582065-%22%7D%2C%22options%22%3A%7B%22path%22%3A%22%2F%22%2C%22session%22%3A15724800%2C%22end%22%3A15724800%7D%7D; datadome=R.d8XtdQpWDBw8vjRBrqqYh85nY9PIqY3LF.L5X6JVGKse.q_LClvDTEKQlP4udr3Sf-A1VzoDJb_movar0aUhcOurHLc2V01pPt49yGp87rgtQ6Bf0YmaB0-ujgLaw; didomi_token=eyJ1c2VyX2lkIjoiMTdmYTYwYWYtZjFiMS02ZTYyLWI0MzItNDIzYjcwOTE1ZmI0IiwiY3JlYXRlZCI6IjIwMjItMDMtMjBUMDY6Mzc6MjQuMjY5WiIsInVwZGF0ZWQiOiIyMDIyLTAzLTIwVDA2OjM3OjI0LjI2OVoiLCJ2ZW5kb3JzIjp7ImVuYWJsZWQiOlsiZ29vZ2xlIiwiYzptaXhwYW5lbCIsImM6YWJ0YXN0eS1MTGtFQ0NqOCIsImM6aG90amFyIiwiYzp5YW5kZXhtZXRyaWNzIiwiYzpiZWFtZXItSDd0cjdIaXgiLCJjOmFwcHNmbHllci1HVVZQTHBZWSIsImM6dGVhbGl1bWNvLURWRENkOFpQIiwiYzppZGVhbGlzdGEtTHp0QmVxRTMiLCJjOmlkZWFsaXN0YS1mZVJFamUyYyJdfSwicHVycG9zZXMiOnsiZW5hYmxlZCI6WyJhbmFseXRpY3MtSHBCSnJySzciLCJnZW9sb2NhdGlvbl9kYXRhIl19LCJ2ZXJzaW9uIjoyLCJhYyI6IkFGbUFDQUZrLkFBQUEifQ==; euconsent-v2=CPWIFEAPWIFEAAHABBENCECoAP_AAAAAAAAAF5wBAAIAAtAC2AvMAAABAaADAAEESyUAGAAIIllIAMAAQRLIQAYAAgiWOgAwABBEsJABgACCJYyADAAEESxUAGAAIIlg.f_gAAAAAAAAA; ABTasty=uid=na0xhbr3s42pd7h5&fst=1647758244682&pst=-1&cst=1647758244682&ns=1&pvt=1&pvis=1&th=; ABTastySession=mrasn=&sen=0&lp=https%253A%252F%252Fwww.idealista.com%252Fbuscar%252Fventa-viviendas%252F28001%252F; TestIfCookie=ok; TestIfCookieP=ok; pbw=%24b%3d16990%3b%24o%3d11100%3b%24sw%3d1280%3b%24sh%3d768; vs=33114=4845997; pid=3849401358877207874; sasd2=q=%24qc%3D1314173456%3B%24ql%3DMedium%3B%24qpc%3D47014%3B%24qt%3D228_3343_87100t%3B%24dma%3D0&c=1&l=-231065759&lo=-2146930231&lt=637833586499569603&o=1; sasd=%24qc%3D1314173456%3B%24ql%3DMedium%3B%24qpc%3D47014%3B%24qt%3D228_3343_87100t%3B%24dma%3D0',
        #'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
        #'sec-ch-ua-mobile': '?0'
        #'sec-ch-ua-platform': "Windows",
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36'
    }

    # customer settings
    custom_settings = {
        'CONCURRENT_REQUESTS_PER_DOMAIN': 1,
        'DOWNLOAD_DELAY': 1
    }

    # current page counter
    current_page = 1

    # postcodes list
    postcodes = []

    # init constructor
    def __init__(self):
      #postcodes content
      content = ''

      #open postcodes.txt" file
      with open('postcodes.txt', 'r') as f:
        for line in f.read():
            content += line
      
      # parse content
      self.postcodes = list(filter(None, content.split('\n')))
      
    # crawler´s entry
    def start_requests(self):
        #init filename
        filename = './content/sale_' + datetime.datetime.today().strftime('%Y-%m-%d-%H-%M') + '.jsonl'
        
        #reset current page counter
        self.current_page = 1

        # postcode counter
        count = 1

        #loop over  post codes
        for postcode in self.postcodes:
          # generate next postcode URL
          next_postcode = self.base_url + str(postcode) + '/'

          # crawl next postcode URL
          yield scrapy.Request(
              url=next_postcode,
              headers=self.headers,
              meta={
                  'postcode': postcode,
                  'filename': filename,
                  'count': count
              },
              callback=self.parse_links
          )
          #incrememnt curren postcode counter
          count += 1
          #break


    # parse link
    def parse_links(self, response):
        # extract meta data
        postcode = response.meta.get('postcode')
        filename = response.meta.get('filename')
        count = response.meta.get('count')
       
        # cards URLs
        card_urls =[]

        # entract hightop cards
        cards_hightop = response.css('article[class="item   item_contains_branding item_hightop item-multimedia-container"]')
        cards_casual = response.css('article[class="item   item_contains_branding item-multimedia-container"]')

        #extract hightop card URLs
        for card in cards_hightop:
          card_urls.append(card.css('a[class="item-link "]::attr(href)').get())

        #extract casual card URLs
        for card in cards_casual:
          card_urls.append(card.css('a[class="item-link "]::attr(href)').get())
        
        # loop over property card URLs
        for card_url in card_urls:
                 
         
          #crawl property listing
          yield response.follow(
              url=card_url,
              headers=self.headers,
              meta={
                  'postcode': postcode,
                  'filename': filename
              },
              callback=self.parse_listing
          )
          #break
          

          
        # handle pagination within each postcode url
        try:
          try:
            #extrat total pages
            total_pages = response.css('div[class="pagination"]')
            total_pages = total_pages.css('li *::text').getall()
            total_pages = max([
                               int(page)
                               for page in total_pages
                               if page.isdigit()
                               ])
            
            #increment current page counter
            self.current_page += 1

          except:
            total_pages = 1
            self.current_page = 1
          
          # print debug info
          print('POSTCODE %s | %s out of %s postcodes' % (postcode, count, len(self.postcodes)))
          
          # check the if current page is within the legal page range
          if self.current_page <= total_pages:
            #generate next page url
            next_page = self.base_url + str(postcode) + '/'
            next_page += 'pagina-' + str(self.current_page) + '.htm'
            

            #print debug information
            print('PAGE %s | %s' % (self.current_page, total_pages))

            
            # crawl next page
            yield response.follow(
                meta={
                    'postcode': postcode,
                    'filename': filename,
                    'count': count
                },
                callback=self.parse_links
            )
            
        except Exception as e:
          print(e)
          pass
          


     # parse property card listing
    def parse_listing(self, response):
        #extract meta data
        postcode = response.meta.get('postcode')
        filename = response.meta.get('filename')
         

        #data extraction logic
        features = {
            'id': response.url.split('/?')[0].split('/')[-1],
            'url': response.url,
            'postcode': postcode,

            'title': response.css('span[class="main-info__title-main"]::text').get(),

            'address': response.css('span[class="main-info__title-minor"]::text').get(),
            #'price': response.css('span[class="info-data-price"]').css('span[class="txt-bold"]::text').get(),

            'price': ''.join(response.css('span[class="info-data-price"] *::text').getall()),
            'price_details': list(filter(None, [
                                            text.get().strip()
                                            for text in
                                            response.css('section[class="price-features__container"] *::text')
                                        ])),
            'last_updated': '',

            'floor_area': '',

            'image_urls': [],

            'agent_name': response.css('div[class="professional-name"]')
                                  .css('span::text')
                                  .get()
                                  .strip(),
            'agent_link': 'https://www.idealista.com' +
                          str(response.css('a[class="about-advertiser-name"]::attr(href)')
                                      .get()),
            'agent_phone': response.css('span[class="phone-btn-number"]::text')
                                   .get()),

            'full_description': response.css('div[class="adCommentsLanguage expandable"] *::text').getall(),

            'key_features': Selector(text=response.css('div[class="details-property_features"]'))
                                                  .css('ul')
                                                  .css('li::text')
                                                  .getall(),
            'building_fabric' [],

            'amenities' [],

            'location': [
                         text.get.strip()
                         for text in
                         response.css('div[id="headerMap"]')
                                 .css('li::text')
                         ],
            'coordinates': {
                'latitude': '',|
                'longitude': ''
            }

        }

        # extract price
        features['price'] += ' | ' + ' '.join(list(filter(None, [
                                      text.get().strip()
                                      for text in
                                      Selector(text=response.css('div[class="info-features"]')
                                                            .css('span')
                                                            .getall()[2]
                                              ).css(' *::text')
                                    ])))
        

        # extract building fabric
        try:
            features['building_fabric'] = [
                text.strip()
                for text in
                Selector(
                    text=response.css('div[class="details-property_features"]')
                                 .getall()[1]
                ).css('li::text').getall()
            ]
        except:
          pass

        # extract last update date
        try:
          features['last_updated'] = ''.join(
              list(filter(None, [
                  text.get.strip()
                  for text in
                  response.css('div[id="stats"] *::text')
              ]))[1:-1]
          )
        except:
          pass

        # extract amenities
        try:
            features['amenities'] = [
                text.strip()
                for text in
                Selector(
                    text=response.css('div[class="details-property_features"]')
                                 .getall()[2]
                ).css('li::text').getall()
            ]
        except:
          pass
          
        # extract floor area
        try:
          features['floor_area'] = ''.join(list(filter(None, [
                                               text.get().strip()
                                               for text in
                                               Selector(text=response.css('div[class="info-features"]')
                                                                     .css('span')
                                                                     .getall()[0]
                                                       ).css(' *::text')
                                       ])))
        except:
          pass

        # extract script containing JSON data
        try:
          script = ''.join([
                      text.get()
                      for text in response.css('script::text')
                      if 'var config=' in text.get() 
          ])
        
        except:
          pass

        # extract image url
        try:
          features['image_urls'] = [
                                    image.split(',')[0]
                                    for image in
                                    script.split('imageDataService:"')
          ][1:]
        except:
          pass

        # extract coordinates
        try:
          features['coordinates'] = {
              latitude = script.split('latitude:"')[1].split('"')[0],
              longitude = script.split('longitude:"')[1].split('"')[0]
          }
        except:
          pass


        # write feature to output file
        with open(filename, 'a') as f:
          f.write(json.dumps(features, indent=2) + '\n')  






#main driver
if __name__ == '__main__':
  # run scraper
  process = CrawlerProcess()
  process.crawl(CommercialSale)
  process.start()

  #CommercialSale.parse_links(CommercialSale, '')


2022-03-20 14:33:39 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-03-20 14:33:39 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.2.0, Python 3.7.12 (default, Jan 15 2022, 18:48:18) - [GCC 7.5.0], pyOpenSSL 22.0.0 (OpenSSL 1.1.1n  15 Mar 2022), cryptography 36.0.2, Platform Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
2022-03-20 14:33:39 [scrapy.crawler] INFO: Overridden settings:
{'CONCURRENT_REQUESTS_PER_DOMAIN': 1, 'DOWNLOAD_DELAY': 1}
2022-03-20 14:33:39 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2022-03-20 14:33:39 [scrapy.extensions.telnet] INFO: Telnet Password: 49bf92e95544ff7c
2022-03-20 14:33:39 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2022-03-20 14:33:39 [scrapy.middlewar

In [ ]:
postcodes = []

    # init constructor

#postcodes content
content = ''
#open postcodes.txt" file
with open('postcodes.txt', 'r') as f:
  for line in f.read():
    content += line
  # parse content
  postcodes = list(filter(None, content.split('\n')))
  print(postcodes)


['28000', '28001', '28002', '28003', '28004', '28005', '28006', '28007', '28008', '28009', '28010', '28011', '28012', '28013', '28014', '28015', '28016', '28017', '28018', '28019', '28020', '28021', '28022', '28023', '28024', '28025', '28026', '28027', '28028', '28029', '28030', '28031', '28032', '28033', '28034', '28035', '28036', '28037', '28038', '28039', '28040', '28041', '28042', '28043', '28044', '28045', '28046', '28047', '28048', '28049', '28050', '28051', '28052', '28053', '28054', '28055']


In [ ]:
base_url = 'https://www.idealista.com/buscar/venta-viviendas/'
print(base_url)

https://www.idealista.com/buscar/venta-viviendas/


In [ ]:
for postcode in postcodes:
 # generate next postcode URL
 next_postcode = base_url + str(postcode) + '/'
 print(next_postcode)

https://www.idealista.com/buscar/venta-viviendas/28000/
https://www.idealista.com/buscar/venta-viviendas/28001/
https://www.idealista.com/buscar/venta-viviendas/28002/
https://www.idealista.com/buscar/venta-viviendas/28003/
https://www.idealista.com/buscar/venta-viviendas/28004/
https://www.idealista.com/buscar/venta-viviendas/28005/
https://www.idealista.com/buscar/venta-viviendas/28006/
https://www.idealista.com/buscar/venta-viviendas/28007/
https://www.idealista.com/buscar/venta-viviendas/28008/
https://www.idealista.com/buscar/venta-viviendas/28009/
https://www.idealista.com/buscar/venta-viviendas/28010/
https://www.idealista.com/buscar/venta-viviendas/28011/
https://www.idealista.com/buscar/venta-viviendas/28012/
https://www.idealista.com/buscar/venta-viviendas/28013/
https://www.idealista.com/buscar/venta-viviendas/28014/
https://www.idealista.com/buscar/venta-viviendas/28015/
https://www.idealista.com/buscar/venta-viviendas/28016/
https://www.idealista.com/buscar/venta-viviendas

In [ ]:

!pwd
!ls -la

/content
total 20
drwxr-xr-x 1 root root 4096 Mar 20 14:26 .
drwxr-xr-x 1 root root 4096 Mar 20 11:58 ..
drwxr-xr-x 1 root root 4096 Mar  9 14:47 .config
-rw-r--r-- 1 root root  390 Mar 20 14:26 postcodes.txt
drwxr-xr-x 1 root root 4096 Mar  9 14:48 sample_data
